На сайте запущен А/В тест с целью увеличить доход. 

В приложенном excel файле вы найдете сырые данные по результатам эксперимента – user_id, тип выборки variant_name и доход принесенный пользователем revenue.

Проанализируйте результаты эксперимента и напишите свои рекомендации менеджеру.

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_1samp


Для начала загрузим и посмотрин на данные:

In [3]:
df = pd.read_excel('AB_Test_Results.xlsx');
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   USER_ID       10000 non-null  int64  
 1   VARIANT_NAME  10000 non-null  object 
 2   REVENUE       10000 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 234.5+ KB


c:\Users\andre\AppData\Local\Programs\Python\Python310\lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [4]:
df.describe()

,USER_ID,REVENUE
count,10000.000000,10000.000000
mean,4981.080200,0.099447
std,2890.590115,2.318529
min,2.000000,0.000000
25%,2468.750000,0.000000
50%,4962.000000,0.000000
75%,7511.500000,0.000000
max,10000.000000,196.010000


In [5]:
df.head()

,USER_ID,VARIANT_NAME,REVENUE
0,737,variant,0.0
1,2423,control,0.0
2,9411,control,0.0
3,7311,control,0.0
4,6174,variant,0.0


 Избавимся от некорретных строк. Посчитаем, сколько типов встречается у каждого пользователя.

In [21]:
v = df.\
    groupby('USER_ID', as_index=False).\
    agg({'VARIANT_NAME': pd.Series.nunique})
v.head()

,USER_ID,VARIANT_NAME
0,2,1
1,3,2
2,4,1
3,5,1
4,6,1


In [20]:
more_than_one_types = v.query('VARIANT_NAME > 1')
df_n = df[~df.USER_ID.isin(more_than_one_types.USER_ID)].sort_values('USER_ID')
df_n.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6070 entries, 3479 to 1064
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   USER_ID       6070 non-null   int64  
 1   VARIANT_NAME  6070 non-null   object 
 2   REVENUE       6070 non-null   float64
dtypes: float64(1), int64(1), object(1)
memory usage: 189.7+ KB


Cравнив разницу длин таблиц видим, что очень много юзеров, что попали в обе группы, что есть очень полхо. Эксперимент явно поставлен не корректно.

Разобьём наши данные на контрольную и тестовую группу:

In [13]:
df_control = df_n[df_n["VARIANT_NAME"] == "control"]
df_control.head()

,USER_ID,VARIANT_NAME,REVENUE
3479,2,control,0.0
7076,2,control,0.0
2406,2,control,0.0
8223,11,control,0.0
3889,11,control,0.0


In [14]:
df_variant = df_n[df_n["VARIANT_NAME"] == "variant"]
df_variant.head()

,USER_ID,VARIANT_NAME,REVENUE
9077,4,variant,0.0
9155,5,variant,0.0
7720,6,variant,0.0
3920,9,variant,0.0
458,15,variant,0.0


In [15]:
print(len(df_control), len(df_variant))

3026 3044


Длина их примерно совпадает.

Рассмотрим эти группы на нормальность.

In [16]:
from scipy.stats import shapiro
alpha = 0.05

st = shapiro(df_control.REVENUE)
print('Distribution is {}normal\n'.format( {True:'not ',
False:''}[st[1] < alpha]));

Distribution is not normal



In [17]:
st = shapiro(df_variant.REVENUE)
print('Distribution is {}normal\n'.format( {True:'not ',
False:''}[st[1] < alpha]));

Distribution is not normal



О нормальности говорить не приходится.

Кинем первый оценочный взгляд на данные в группах.

In [18]:
df_control['REVENUE'].describe()

count    3026.000000
mean        0.155506
std         3.708620
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max       196.010000
Name: REVENUE, dtype: float64

In [19]:
df_variant['REVENUE'].describe()

count    3044.000000
mean        0.058909
std         0.760344
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max        23.040000
Name: REVENUE, dtype: float64

Видно, что тестовая группа гораздо хуже контрольной.

Эксперимент неточен, требует доработки и нового запуска. Если предположить, что точен, то в итоге имеем гораздо меньшую стоймость заказа, что негативно сказывается на доходе.